In [2]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt

In [3]:
def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
    print(len(pallet))
trainentest = load_data("demand_kWtrain_val.csv")
base_ft = 



1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
4925535


# Features


## Feature 1 - Temperature and humidity
### What needs to be done
#### preprocess weather dataframe
#### Preprocess base_ft dataframe

In [8]:
def addtimecol(df, colname): ####input df and colname 
        df[colname] = pd.to_datetime(df[colname])         
        df['year'] = df[colname].dt.year
        df['month'] = df[colname].dt.month
        df['weekday'] = df[colname].dt.weekday
        df['day'] = df[colname].dt.day
        df['hour'] = df[colname].dt.hour
        df['minute'] = df[colname].dt.minute        
        return df
def make_datetime(df, col_name): ###To make datetime object from df at colname    
    for index, row in df.iterrows():
        t = df.at[index, col_name]
        time = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
        localtime = time - (timedelta(hours=6))
        df.at[index, 'localstrptime'] = localtime

This is already done once

In [ ]:
#Add localstrptime, 6 hours before UTC
#base_weather = make_datetime(base_weather, 'datetime_UTC')
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])

#Remove unnecessary columns/change column names
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1', 'hour'], axis=1)
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})

#Add time columns.
base_weather = addtimecol(base_weather, 'datetime_local')

In [17]:
base_ft = load_data('featuretrain.csv')
base_ft = addtimecol(base_ft, 'datetime_local')
base_ft['datetime_local'] = pd.to_datetime(base_ft['datetime_local'])
base_ft
type(base_ft['datetime_local'].iloc[3])

pandas._libs.tslibs.timestamps.Timestamp


## Feature 2 - weight coming into the warehouse in the previous 24 h
### What needs to be done
#### preprocess inbound dataframe
drop missing weight values

fill in either load_start_datetime or truck_signin time as start of 24 hours

Make the index the datetime and sort it

#### create new_ft dataframe to add the feature to

#### create a dataframe to combine all resampled data into 



Preprocessing inbound dataframe
I decided to drop missing load_start_datetime, ideal would be to fill in truck_signin_datetime when load_starttime is missing

In [141]:
inbound = load_data("inbound_loads.csv")
inbound_sec = inbound.dropna(subset = ['net_weight', 'load_start_datetime'])
#inbound_sec = inbound_sec.dropna(axis = 0, how = 'any', thresh = 1, subset = ['load_start_datetime', 'truck_signin_datetime'], inplace = True)
inbound_sec = inbound_sec.drop('Unnamed: 0', axis = 1)
inbound_sec = inbound_sec.sort_values('load_start_datetime')
inbound_sec['datetime_local'] = pd.to_datetime(inbound_sec['load_start_datetime'])
inbound_sec = inbound_sec.set_index('datetime_local')



In [142]:
inbound_sec

,warehouse_order_number,customer_code,load_reference_number,carrier_code,front_temperature,middle_temperature,back_temperature,net_weight,weight_uom,case_quantity,pallet_count,load_start_datetime,load_finish_datetime,signout_datetime,dock_door_number,trailer_number,truck_signin_datetime
datetime_local,,,,,,,,,,,,,,,,,
2018-12-30 06:33:00,1315527.0,899.0,I42168974,CHEF,-5.0,-6.0,-6.0,42796.0,pounds,1950.0,24.0,2018-12-30 06:33:00,2018-12-30 09:38:00,2018-12-30 05:38:15,1.0,U69689,2018-12-30 05:38:14
2018-12-31 11:45:00,1315640.0,7240.0,I42169019,TYSF,0.0,0.0,0.0,900.0,pounds,60.0,2.0,2018-12-31 11:45:00,2018-12-31 11:51:00,2018-12-31 09:48:50,44.0,29261,2018-12-31 09:48:49
2018-12-31 14:01:00,1312923.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,8640.0,pounds,288.0,4.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312927.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,1170.0,pounds,39.0,1.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312928.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,2940.0,pounds,98.0,1.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312932.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,6690.0,pounds,223.0,3.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312934.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,5850.0,pounds,195.0,2.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312935.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,270.0,pounds,9.0,1.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312940.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,15596.0,pounds,520.0,7.0,2018-12-31 14:01:00,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44


In [22]:
new_ft = base_ft.copy()
new_ft['datetime_local'] = pd.to_datetime(new_ft['datetime_local'])
new_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


In [143]:
startingpoint = new_ft['datetime_local'].iloc[0]
inbound_resampled_data_samen = inbound_sec.net_weight.resample('1380Min', base = 0, closed = 'right', label = 'right', kind = 'timestamp').sum()


for minute in range(0,1380):    
    df2 = inbound_sec.net_weight.resample('1380Min', base = minute, closed = 'right',label = 'right', kind='timestamp').sum() 
    inbound_resampled_data_samen = pd.concat([inbound_resampled_data_samen, df2])
    
inbound_resampled_data_samen = inbound_resampled_data_samen = inbound_resampled_data_samen.sort_index()

Paste inbound_resampled_data_samen onto the feature df
First get both indexes the same

In [229]:
inbound_resampled_df = inbound_resampled_data_samen.to_frame()
inbound_resampled_df['weight_23h'] = inbound_resampled_df['net_weight']
inbound_resampled_df = inbound_resampled_df.drop(['net_weight'], axis = 1)
inbound_resampled_df.reset_index(inplace = True)
inbound_resampled_df['datetime_local'] = pd.to_datetime(inbound_resampled_df['datetime_local'])
inbound_resampled_df['datetime_local'] = pd.Series(inbound_resampled_df.datetime_local.dt.to_pydatetime(), dtype='O')

inbound_resampled_df

,datetime_local,weight_23h
0,2018-12-30 06:33:00,42796.0
1,2018-12-30 06:34:00,42796.0
2,2018-12-30 06:35:00,42796.0
3,2018-12-30 06:36:00,42796.0
4,2018-12-30 06:37:00,42796.0
5,2018-12-30 06:38:00,42796.0
6,2018-12-30 06:39:00,42796.0
7,2018-12-30 06:40:00,42796.0
8,2018-12-30 06:41:00,42796.0
9,2018-12-30 06:42:00,42796.0


In [217]:
new_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


In [234]:
update_ft = new_ft.copy()
update_ft['datetime_local'] = pd.to_datetime(update_ft['datetime_local'])
update_ft['datetime_local'] = pd.Series(update_ft.datetime_local.dt.to_pydatetime(), dtype='O')
update_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


In [232]:
#newest_ft = update_ft.merge(inbound_resampled_df, how='left', on='datetime_local')
weightin_merged_df = update_ft.merge(inbound_resampled_df, left_on='datetime_local', right_index=True, how='left').reset_index()
newest_ft['weight_23h'] = newest_ft['weight_23h'].fillna(0)

ValueError: You are trying to merge on datetime64[ns] and int64 columns. If you wish to proceed you should use pd.concat

In [235]:
update_ft = update_ft.set_index('datetime_local')
inbound_resampled_df = inbound_resampled_df.set_index('datetime_local')
concated_df = pd.concat([update_ft, inbound_resampled_df], axis = 1)

In [236]:
merge_df

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,net_weight
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00,295346.0
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00,295346.0
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00,295346.0
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00,295346.0
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00,295346.0
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00,295346.0
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00,295346.0
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00,295346.0
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00,295346.0
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00,295346.0



## Feature 3 - pallets moved in the previous 5 minutes
### What needs to be done
#### preprocess pallet dataframe
drop missing pallet values

Make the index the datetime and sort it

#### create new_ft dataframe to add the feature to

#### create a dataframe to combine all resampled data into 



In [174]:
pallet_sec = pallet.copy()
#pallet_sec = pallet_sec[pallet_sec['quantity'] != 0]
pallet_sec = pallet_sec.drop(['Unnamed: 0','tran_type', 'Unnamed: 0.1', 'source_system_id', 'warehouse_facility_id', 'final_pallet_code', 'lot_code'], axis = 1)

pallet_sec['transaction_date'] = pd.to_datetime(pallet_sec['transaction_date'])
pallet_sec = pallet_sec.reset_index()

#remove previous not-working index
pallet_sec = pallet_sec.drop(['index'], axis = 1)

#Make datetime_local a datetime and its index
pallet_sec['datetime_local'] = pd.to_datetime(pallet_sec['transaction_date'])
pallet_sec = pallet_sec.sort_values('datetime_local')
pallet_sec.set_index('datetime_local', inplace = True)

In [187]:
pallet_new_ft = newest_ft.copy()
pallet_new_ft['datetime_local'] = pd.to_datetime(pallet_new_ft['datetime_local'])
pallet_new_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,net_weight
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00,295346.0
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00,295346.0
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00,295346.0
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00,295346.0
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00,295346.0
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00,295346.0
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00,295346.0
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00,295346.0
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00,295346.0
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00,295346.0


In [189]:
startingpoint = pallet_new_ft['datetime_local'].iloc[0]
pallet_resampled_data = pallet_sec.quantity.resample('5Min', base = 0, closed = 'right', label = 'right', kind = 'timestamp').sum()


for minute in range(0,5):    
    df2 = pallet_sec.quantity.resample('5Min', base = minute, closed = 'right',label = 'right', kind='timestamp').sum() 
    pallet_resampled_data = pd.concat([pallet_resampled_data, df2])
    
pallet_resampled_data = pallet_resampled_data.sort_index()

In [192]:
pallet_resampled_df = pallet_resampled_data.to_frame()
pallet_resampled_df.reset_index(inplace = True)
pallet_resampled_df['datetime_local'] = pd.to_datetime(pallet_resampled_df['datetime_local'])
pallet_resampled_df['datetime_local'] = pd.Series(pallet_resampled_df.datetime_local.dt.to_pydatetime(), dtype='O')

pallet_resampled_df


,datetime_local,quantity
0,2019-01-02 00:30:00,63
1,2019-01-02 00:30:00,63
2,2019-01-02 00:31:00,63
3,2019-01-02 00:32:00,126
4,2019-01-02 00:33:00,126
5,2019-01-02 00:34:00,189
6,2019-01-02 00:35:00,189
7,2019-01-02 00:35:00,189
8,2019-01-02 00:36:00,189
9,2019-01-02 00:37:00,189


In [205]:
pallet_update_ft = pallet_new_ft.copy()

pallet_update_ft['datetime_local'] = pd.to_datetime(pallet_update_ft['datetime_local'])
pallet_update_ft['datetime_local'] = pd.Series(pallet_update_ft.datetime_local.dt.to_pydatetime(), dtype='O')


,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,net_weight
20000,2019-03-04 13:00:00,2019,3,0,4,13,0,2019-03-04 13:00:00-06:00,53.94,26.60,2019-03-04 19:00:00,759011.0
20001,2019-03-04 13:05:00,2019,3,0,4,13,5,2019-03-04 13:05:00-06:00,49.79,26.60,2019-03-04 19:05:00,759011.0
20002,2019-03-04 13:10:00,2019,3,0,4,13,10,2019-03-04 13:10:00-06:00,49.79,26.60,2019-03-04 19:10:00,759011.0
20003,2019-03-04 13:15:00,2019,3,0,4,13,15,2019-03-04 13:15:00-06:00,49.79,26.60,2019-03-04 19:15:00,759011.0
20004,2019-03-04 13:20:00,2019,3,0,4,13,20,2019-03-04 13:20:00-06:00,49.79,26.60,2019-03-04 19:20:00,759011.0
20005,2019-03-04 13:25:00,2019,3,0,4,13,25,2019-03-04 13:25:00-06:00,49.79,26.60,2019-03-04 19:25:00,857135.0
20006,2019-03-04 13:30:00,2019,3,0,4,13,30,2019-03-04 13:30:00-06:00,49.79,26.60,2019-03-04 19:30:00,857135.0
20007,2019-03-04 13:35:00,2019,3,0,4,13,35,2019-03-04 13:35:00-06:00,53.94,26.60,2019-03-04 19:35:00,857135.0
20008,2019-03-04 13:40:00,2019,3,0,4,13,40,2019-03-04 13:40:00-06:00,49.79,26.60,2019-03-04 19:40:00,894513.0
20009,2019-03-04 13:45:00,2019,3,0,4,13,45,2019-03-04 13:45:00-06:00,49.79,26.60,2019-03-04 19:45:00,894513.0


In [208]:
pallet_newest_ft = pallet_update_ft.merge(pallet_resampled_df, how='left', on='datetime_local')

In [209]:
pallet_newest_ft[20000:20500]

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,net_weight,quantity
20000,2019-02-03 08:40:00,2019,2,6,3,8,40,2019-02-03 08:40:00-06:00,93.80,60.80,2019-02-03 14:40:00,796170.0,858.0
20001,2019-02-03 08:40:00,2019,2,6,3,8,40,2019-02-03 08:40:00-06:00,93.80,60.80,2019-02-03 14:40:00,796170.0,858.0
20002,2019-02-03 08:45:00,2019,2,6,3,8,45,2019-02-03 08:45:00-06:00,87.94,60.80,2019-02-03 14:45:00,764123.0,1198.0
20003,2019-02-03 08:45:00,2019,2,6,3,8,45,2019-02-03 08:45:00-06:00,87.94,60.80,2019-02-03 14:45:00,764123.0,1198.0
20004,2019-02-03 08:50:00,2019,2,6,3,8,50,2019-02-03 08:50:00-06:00,93.80,60.80,2019-02-03 14:50:00,764123.0,1432.0
20005,2019-02-03 08:50:00,2019,2,6,3,8,50,2019-02-03 08:50:00-06:00,93.80,60.80,2019-02-03 14:50:00,764123.0,1432.0
20006,2019-02-03 08:53:00,2019,2,6,3,8,53,2019-02-03 08:53:00-06:00,93.20,60.98,2019-02-03 14:53:00,764123.0,855.0
20007,2019-02-03 08:55:00,2019,2,6,3,8,55,2019-02-03 08:55:00-06:00,87.94,60.80,2019-02-03 14:55:00,764123.0,1496.0
20008,2019-02-03 08:55:00,2019,2,6,3,8,55,2019-02-03 08:55:00-06:00,87.94,60.80,2019-02-03 14:55:00,764123.0,1496.0
20009,2019-02-03 09:00:00,2019,2,6,3,9,0,2019-02-03 09:00:00-06:00,88.02,62.60,2019-02-03 15:00:00,737365.0,1258.0


In [204]:
pallet_newest_ft['quantity'] = pallet_newest_ft['quantity'].fillna(0)
pallet_newest_ft[20000:20500]

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,net_weight,quantity
20000,2019-02-03 08:40:00,2019,2,6,3,8,40,2019-02-03 08:40:00-06:00,93.80,60.80,2019-02-03 14:40:00,796170.0,858.0
20001,2019-02-03 08:40:00,2019,2,6,3,8,40,2019-02-03 08:40:00-06:00,93.80,60.80,2019-02-03 14:40:00,796170.0,858.0
20002,2019-02-03 08:45:00,2019,2,6,3,8,45,2019-02-03 08:45:00-06:00,87.94,60.80,2019-02-03 14:45:00,764123.0,1198.0
20003,2019-02-03 08:45:00,2019,2,6,3,8,45,2019-02-03 08:45:00-06:00,87.94,60.80,2019-02-03 14:45:00,764123.0,1198.0
20004,2019-02-03 08:50:00,2019,2,6,3,8,50,2019-02-03 08:50:00-06:00,93.80,60.80,2019-02-03 14:50:00,764123.0,1432.0
20005,2019-02-03 08:50:00,2019,2,6,3,8,50,2019-02-03 08:50:00-06:00,93.80,60.80,2019-02-03 14:50:00,764123.0,1432.0
20006,2019-02-03 08:53:00,2019,2,6,3,8,53,2019-02-03 08:53:00-06:00,93.20,60.98,2019-02-03 14:53:00,764123.0,855.0
20007,2019-02-03 08:55:00,2019,2,6,3,8,55,2019-02-03 08:55:00-06:00,87.94,60.80,2019-02-03 14:55:00,764123.0,1496.0
20008,2019-02-03 08:55:00,2019,2,6,3,8,55,2019-02-03 08:55:00-06:00,87.94,60.80,2019-02-03 14:55:00,764123.0,1496.0
20009,2019-02-03 09:00:00,2019,2,6,3,9,0,2019-02-03 09:00:00-06:00,88.02,62.60,2019-02-03 15:00:00,737365.0,1258.0
